<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/GDrive_File_Utilities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %%writefile GDriveFileUtilities.py
# 08/18/2020-1
from __future__ import absolute_import
from IPython.display import clear_output
import glob, os, pathlib, shutil, sys
from os.path import *
from pathlib import Path
from google.colab import drive, files

global C
global fu

class ColorOutput:
    LITE = '\033[96m'; HEADER = '\033[95m'; BLUE = '\033[94m';
    WARNING = '\033[93m'; GREEN = '\033[92m'; RED = '\033[91m'; 
    DARK = '\033[90m'; UNDERLINE = '\033[4m';BOLD = '\033[1m'; 
    WHITE = '\033[0m'

C = ColorOutput()

class FileUtilities(ColorOutput):
    __all__ = sorted(['C', 'allFiles', 'allFilesList', 'allFilesDict',
                      'cwdPath', 'homePath','inspectFile', 'listFiles',
                      'printSystemPaths','setSystemPaths','updateAllFiles', 
                      'mountGoogleDrive', 'deleteGDriveFile', 
                      'searchGDriveFiles', 'foundFilesDict','changeDir', 
                      'writeToGoogleDrive', 'localPathsDict', 'fullWalkList'])

    def __init__(self):
        self.homePath = Path.home()
        self.cwdPath = Path.cwd()
        self.allFiles = glob.glob('**', recursive=True)
        self.allFilesList = self.allFiles

        self.fullWalkList = []
        self.allFilesDict = {}
        self.foundFilesDict = {}
        self.localPathsDict = {}

    def mountGoogleDrive(self):
        drive.mount('gdrive', force_remount=True, timeout_ms=600000)

    def updateAllFiles(self):
        os.chdir("/content/")
        self.allFiles = glob.glob('**', recursive=True)

    def setSystemPaths(self, addNewPath=None):
        addNewPath = realpath(addNewPath)
        pathList = sys.path.copy()
        if not addNewPath in pathList and exists(addNewPath) and addNewPath != 'None':
            sys.path.append(addNewPath)

    def listFiles(self, altFileList=[], silent=False):
        self.updateAllFiles()
        if altFileList == []:
            pass
        else:
            self.allFiles = altFileList

        if not silent:
            print(f'{C.BLUE}')
        self.allFiles = sorted(self.allFiles)
        tempDict = self.allFilesDict
        fileCount = 0
        if not silent:
            print(f'{C.BLUE}{self.cwdPath}')
        self.setSystemPaths(addNewPath=self.cwdPath)
        for fil in self.allFiles:
            fullPath = Path(realpath(fil)) # POSIX path
            realPath = realpath(fil) # String path
            if isdir(fullPath) and not realPath.__contains__('__pycache__'):  
                if not silent:
                    print(f'\n{C.BLUE}{fullPath}')
                self.setSystemPaths(addNewPath=realPath) 
            elif isfile(fullPath) and not realPath.__contains__('__pycache__'): 
                fileCount += 1
                if not silent:
                    print(f'{C.WHITE}{fileCount}. {fullPath.name}') 
                tempDict[fileCount] = fullPath
                self.allFilesDict = {}
                self.allFilesDict.update(tempDict)

    def inspectFile(self, selection):
        if selection <= len(self.allFilesDict):  
            try:
                pth = self.allFilesDict[selection]  
                with open(pth, 'r') as openFile:  
                    print(f'# %%writefile {pth.name}\n')  
                    print(openFile.read())
            except:
                print(f'{C.RED}BULLSHIT')

    def printSystemPaths(self):
        pathList = sys.path.copy() 
        pathCount = 0
        for pth in sorted(pathList):
            pathCount += 1
            print(f'{C.WHITE}{pathCount}. {C.BLUE}{pth}')

    def searchGDriveFiles(self):
        os.chdir("/content/")
        print(f'search in: {C.BLUE}{Path.cwd()}{C.WHITE}')
        searchFilesList = glob.glob('**', recursive=True)
        searchTerm = str(input('for: ' ))
        fileCounter = 0
        tempDict = {}
        for fil in sorted(searchFilesList):
            if fil.__contains__(searchTerm) and not fil.__contains__('__pycache__'):
                fullPath = Path(realpath(fil))
                thisPath = realpath(fil)
                if isfile(fullPath):
                    fileCounter += 1
                    tempDict[fileCounter] = fullPath
        self.foundFilesDict = tempDict.copy()
        for item in self.foundFilesDict:
            print(f'{C.WHITE}{item}. {self.foundFilesDict[item].name}') 
            print(f'{C.BLUE}{self.foundFilesDict[item]}\n')

    def deleteGDriveFile(self):
        self.searchGDriveFiles()
        print(f'{C.RED}')
        if len(self.foundFilesDict) > 0:
            selectedFile = int(input('delete file:' ))
            filePath = self.foundFilesDict[selectedFile]
            if exists(filePath) and isfile(filePath) and not isdir(filePath): 
                thisPath = realpath(filePath)
                dirPath = os.path.split(thisPath)
                os.chdir(dirPath[0])
                # %cd dirPath[0]
                # print(f'{C.DARK}{dirPath}')
                print(f'\n{C.HEADER}{filePath}{C.WHITE}\n')
                %rm filePath
        else: print(f'search term not found')
            
    def changeDir(self):
        sysPaths = sys.path.copy()
        pathDict = {}
        counter = 0
        thisRange:range
        for pth in sorted(sysPaths):
            if pth.__contains__('/content'):
                pth = pth + '/'
                counter += 1
                pathDict[counter] = pth
            else: pass

        self.localPathsDict = pathDict.copy()
        thisRange = range(1, len(self.localPathsDict))
        for item in sorted(self.localPathsDict):
            print(f'{C.WHITE}{item}. {C.HEADER}{self.localPathsDict[item]}')
        print(f'\n{C.GREEN}{Path.cwd()}')

        choice = int(input('choice: '))
        if choice in thisRange:
            clear_output()
            os.chdir(self.localPathsDict[choice])
            return self.localPathsDict[choice]
        else: print(f'{C.RED}must be in {thisRange}{C.WHITE}')

    def writeToGoogleDrive(self):
        selectionRange = range(1, len(self.allFilesDict))
        print(selectionRange)
        try:
            selectedFileNumber = int(input('selected file: ' ))
            if selectedFileNumber in range(1, len(self.allFilesDict)):
                pth = realpath(fu.allFilesDict[selectedFileNumber])
                writeToPath = None
                print(f'{C.GREEN}selecter file:{pth}')
                print(f'{C.BLUE}cwd: {os.getcwd()}')
            else: pass
        except Exception as err:
            print(f'{C.RED}{err}')

fu = FileUtilities()
fu.mountGoogleDrive()
fu.listFiles(silent=True)
print(f'{C.GREEN}Ok')

In [ ]:
unknownDict={}
def walkGoogleDrive(fullWalkList=fu.fullWalkList, silent=True):
    walkDirectory = fu.changeDir()
    print(walkDirectory)
    # os.chdir('/content/')
    path = realpath(Path.cwd())
    dirList=[]; fileList=[]
    count=0; dirCount=0; fileCount=0
    st:os.stat

    if not silent:
        print(f'{C.HEADER}cwdPath: {path}')
    for (dirPath, dirNames, fileNames) in os.walk(path):
        dirList.extend(dirNames)
        fileList.extend(fileNames)

    fu.fullWalkList = dirList + fileList
    if not silent:
        print(len(fu.fullWalkList))

    for item in sorted(fu.fullWalkList):
        item = realpath(item)
        posixItem = Path(item)
        if exists(posixItem) and isdir(posixItem):
            st = os.stat(posixItem)
            dirCount+=1
            if not silent:
                print(f'\n{C.BLUE}{item}')

        elif exists(posixItem) and isfile(posixItem):
            st = os.stat(posixItem)
            fileCount+=1
            if not silent:
                print(f'{C.WHITE}{fileCount}. {posixItem.name}')

        elif not isdir(posixItem) and not isfile(posixItem):
            count+=1
            unknownDict[count] = posixItem
            if not silent:
                print(f'{C.GREEN}{item}')
                # print(f'{C.WHITE}ok')

walkGoogleDrive(silent=True)

In [ ]:

print(Path.cwd())
walkGoogleDrive(silent=False)


In [ ]:
for val in unknownDict.values():
    print(f'{C.LITE}{val.owner.__class__.__name__}')
    print(f'{C.BLUE}{val}{C.LITE}')
    print(val.owner.__str__)
    # print(f'__{val.owner.__class__.__text_signature__}')

    print()
    par = val.stat
    par